### **_Import Package_**

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset

import numpy as np
import pandas as pd

import spacy
nlp = spacy.load("en_core_web_sm")

from tqdm import tqdm

In [ ]:
global device
global vocab_size
global bert_model_hidden_size
global lstm_hidden_size
global num_layers
global batch_size

In [ ]:
# global variable iinitialize
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# print(torch.cuda.get_device_name())

bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = BertModel.from_pretrained(bert_model_name).to(device)

vocab_size = bert_model.config.vocab_size
bert_model_hidden_size = bert_model.config.hidden_size
lstm_hidden_size = 256
num_layers = 4
batch_size = 8

### **_Encoder_**

In [ ]:
class Encoder(nn.Module):

    def __init__(self, bert_model):
        super(Encoder, self).__init__()
        self.bert = bert_model
        
    def forward(self, input_ids, attention_mask):
        # with torch.no_grad():
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        return last_hidden_state


### **_Decoder_**

In [ ]:
class Decoder(nn.Module):

    def __init__(self, lstm_hidden_size, num_layers, vocab_size):
        super(Decoder, self).__init__()
        self.lstm = nn.LSTM(input_size=bert_model.config.hidden_size,
                            hidden_size=lstm_hidden_size,
                            num_layers=num_layers).to(device)
        self.linear1 = nn.Linear(lstm_hidden_size, lstm_hidden_size * 2).to(device)
        self.linear2 = nn.Linear(lstm_hidden_size * 2, lstm_hidden_size * 4).to(device)
        self.linear3 = nn.Linear(lstm_hidden_size * 4, vocab_size).to(device)
    
    def forward(self, input):
        lstm_outputs, _ = self.lstm(input)
        output = self.linear1(lstm_outputs)
        output = self.linear2(output)
        prediction = self.linear3(output)
        return prediction
    

### **_Seq2Seq_**

In [ ]:
class Seq2Seq(nn.Module):
    
    def __init__(self, encoder, decoder) -> None:
        '''
        '''
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, input_ids, attention_mask):
        last_hidden_state = self.encoder(input_ids, attention_mask)
        decoder_output = self.decoder(last_hidden_state)
        return decoder_output
        

In [ ]:
class Seq2Seqwithattn(nn.Module):

    def __init__(self, encoder, decoder, attn_embed_dim, num_heads) -> None:
        super(Seq2Seqwithattn, self).__init__()
        self.encoder = encoder
        self.multihead_attn = nn.MultiheadAttention(attn_embed_dim, num_heads).to(device)
        self.decoder = decoder

    def forward(self, input_ids, attention_mask, target_ids):
        encoder_outputs  = self.encoder(input_ids, attention_mask)
        target_outputs = self.encoder(target_ids, attention_mask)
        attn_output, _ = self.multihead_attn(target_outputs,
                                             encoder_outputs,
                                             encoder_outputs)
        decoder_outputs = self.decoder(attn_output)
        return decoder_outputs

In [ ]:
class bertsformer(nn.Module):

    def __init__(self, encoder, bert_model_hidden_size) -> None:
        super(bertsformer, self).__init__()
        self.bert_encoder = encoder
        self.decoder_layer  = nn.TransformerDecoderLayer(d_model=bert_model_hidden_size, nhead=8).to(device)
        self.trans_decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=6, norm=nn.LayerNorm(bert_model_hidden_size)).to(device)
        self.output_layer = nn.Linear(bert_model_hidden_size, vocab_size).to(device)
        
    def forward(self, input_ids, attention_mask, target_ids):
        input_hidden_state = self.bert_encoder(input_ids=input_ids, attention_mask=attention_mask)
        target_hidden_state = self.bert_encoder(input_ids=input_ids, attention_mask=attention_mask)
        
        trans_decoder_output = self.trans_decoder(target_hidden_state, input_hidden_state)
        output = self.output_layer(trans_decoder_output)
        return output

    

### **_Preprocess data_**

In [ ]:
def preprocess_data(text):
    return [tok.text for tok in nlp.tokenizer(text)]

In [ ]:
class PubMedDataset(Dataset):
    
    def __init__(self, df, tokenizer) -> None:
        super().__init__()
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        input_seq = self.df.loc[index, "input"]
        target_seq = self.df.loc[index, "target"]

        encoded_inputs = self.tokenizer.encode_plus(
            input_seq,
            add_special_tokens=True,
            padding='max_length', 
            max_length=128,
            truncation=True,
            return_tensors='pt'
        )
        encoded_targets = self.tokenizer.encode_plus(
            target_seq,
            add_special_tokens=True,
            padding='max_length',
            max_length=128,
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoded_inputs['input_ids'].squeeze(0)
        attention_mask = encoded_inputs['attention_mask'].squeeze(0)
        target_ids = encoded_targets['input_ids'].squeeze(0)

        return input_ids, attention_mask, target_ids

In [ ]:
import json

training_set = r'./../spider/meddialog/results/eval_wer_json/pubmed_46374_train.json'
with open(training_set, 'r') as td:
    data = json.load(td)

In [ ]:
utterances = []
results = []

for d in data:
    utterances.append(d['utterances']['pubmed'])
    results.append(d['results']['pubmed'][2:])

print(len(utterances))

pubmed_df = pd.DataFrame({'input': utterances, 'target': results})
PD = PubMedDataset(pubmed_df, bert_tokenizer)
dataloader = DataLoader(PD, batch_size=batch_size, shuffle=True)

print(pubmed_df.head())

### **_Train model_**

In [ ]:
def train_model(model, criterion, optimizer, dataloader, num_epochs, clip):
    
    # batch accumulation parameter
    accum_iter = 32
    
    model.train()
    
    for epoch in range(num_epochs):
        for batch_idx, (input_ids, attention_mask, target_ids) in tqdm(enumerate(dataloader), desc=f'Epoch {epoch + 1}', ncols=30):
            # optimizer.zero_grad()

            # 將資料移到 device
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            target_ids = target_ids.to(device)
            
            # passes and weights update
            with torch.set_grad_enabled(True):
                
                # forward
                outputs = model(input_ids, attention_mask, target_ids)
                loss = criterion(outputs.view(-1, vocab_size), target_ids.view(-1))

                # normalize loss to account for batch accumulation
                loss = loss / accum_iter

                # backward & optimization
                loss.backward()
                
                torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

                if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(dataloader)):
                    optimizer.step()
                    optimizer.zero_grad()

                    # loss = criterion(outputs.view(-1, vocab_size), target_ids.view(-1))
                    # if ((batch_idx + 1) % 1000 == 0):
                        # print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item()}")
        loss = criterion(outputs.view(-1, vocab_size), target_ids.view(-1))
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

In [ ]:
encoder = Encoder(bert_model=bert_model)
decoder = Decoder(lstm_hidden_size=lstm_hidden_size, num_layers=num_layers, vocab_size=vocab_size)
# bert_lstm_s2s_model = Seq2Seq(encoder, decoder)
seq2seq_with_attn_model = Seq2Seqwithattn(encoder, decoder, bert_model.config.hidden_size, 8)
# bertsformer = bertsformer(encoder, bert_model_hidden_size=bert_model_hidden_size)

# SRC_PAD_IDX = bert_tokenizer.pad_token_id
# TRG_PAD_IDX = bert_tokenizer.pad_token_id

# criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(bert_lstm_s2s_model.parameters(), lr=0.001)
# optimizer = torch.optim.AdamW(bert_lstm_s2s_model.parameters(), lr=0.001)
optimizer = torch.optim.AdamW(seq2seq_with_attn_model.parameters(), lr=0.00008)
# optimizer = torch.optim.AdamW(bertsformer.parameters(), lr=0.00001)

train_model(seq2seq_with_attn_model, \
            criterion=criterion, \
            optimizer=optimizer, \
            dataloader=dataloader, \
            num_epochs=15,
            clip=1)

# torch.save(model.state_dict(), 'model.pth')

### **_Evaluate the model_**

In [ ]:
predicted_sentences = []
bert_lstm_s2s_model.eval()

with torch.no_grad():
    for input_ids, attention_mask, target_ids in dataloader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        outputs = bert_lstm_s2s_model(input_ids, attention_mask)

        _, predicted_ids = torch.max(outputs, dim=2)

        for ids in predicted_ids:
            tokens = bert_tokenizer.convert_ids_to_tokens(ids, skip_special_tokens=True)
            sentence = bert_tokenizer.convert_tokens_to_string(tokens)
            predicted_sentences.append(sentence)


print(predicted_sentences[:10])
